In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

In [ ]:
# Path to the ChromeDriver executable
driver_path = 'D:\\hp5cd\\New folder\\Web_Scraping\\chromedriver-win64\\chromedriver.exe'

# Create a Service object with the path to the ChromeDriver
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service)

In [ ]:
# The URL of the website, with the page number included
url = f'https://maharera.maharashtra.gov.in/projects-search-result?page=1'

# Navigate to the page
driver.get(url)

In [ ]:
# Get the page source
html = driver.page_source

# Parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Total number of results
results = (soup.find('span', class_='colorBlue')).text.strip()
results = int(results)

# As the number of results is too big, so we define it by ourselves for now
results = 400

# Deriving the number of pages you want to scrape
numOfPages = ( results + 9 ) // 10

In [ ]:
# Define a function that will handle the scraping of one page
def scrape_page(page_num):
    # The URL of the website, with the page number included
    url = f'https://maharera.maharashtra.gov.in/projects-search-result?page={page_num}'

    # Navigate to the page
    driver.get(url)

    # Get the page source
    html = driver.page_source

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all divs
    divs = soup.find_all('div', class_='rounded')

    # Initialize an empty list to store the data
    data = []

    # Loop through each div
    for div in divs:
        # Find the link to redirect to the "View details" page in the div
        link = div.find('a', class_='click-projectmodal')

        if link is not None:  # Make sure a link was found
            href = link['href']

            # Navigate to the page
            driver.get(href)

            # Get the page source
            html = driver.page_source

            # Parse the HTML content of the page with BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Extract the information you want from the new page
            infos = soup.find_all('div', class_='form-group')

            # Create a dictionary to store this row of data
            row = {}

            # Loop through each info
            for info in infos:
                # Find all divs within the info
                divs = info.find_all('div')

                if(len(divs) % 2 == 1):
                    # Loop through each pair of divs (key and value)
                    for i in range(0, len(divs) - 2, 2):
                        # Get the key from the label in the first div and value from the second div
                        
                        if 'IndOccu' in divs[i].get('id', ''):
                            divs1 = divs[i].find_all('div')
                            key = divs1[0].text.strip()
                            value = divs1[1].text.strip()
                            row[key] = value
                            break
                            
                        if 'clearfix' in divs[i].get('class', []) and  i == 0:
                            key = divs[1].text.strip()
                            value = divs[2].text.strip()
                            row[key] = value
                            break
                            
                        key = divs[i].text.strip()
                        value = divs[i + 1].text.strip()

                        # Add the key-value pair to the dictionary
                        if (key.endswith(":") or value.endswith(":")):
                            continue
                        else:
                            row[key] = value

                else:
                    for i in range(0, len(divs) - 1, 2):
                        # Get the key from the label in the first div and value from the second div
                        
                        if 'IndOccu' in divs[i].get('id', ''):
                            divs1 = divs[i].find_all('div')
                            key = divs1[0].text.strip()
                            value = divs1[1].text.strip()
                            row[key] = value
                            break
                            
                        if 'clearfix' in divs[i].get('class', []) and  i == 0:
                            key = divs[1].text.strip()
                            value = divs[2].text.strip()
                            row[key] = value
                            break
                            
                        key = divs[i].text.strip()
                        value = divs[i + 1].text.strip()

                        # Add the key-value pair to the dictionary
                        if (key.endswith(":") or value.endswith(":")):
                            continue
                        else:
                            row[key] = value

            # Add the dictionary to the list
            data.append(row)

    return data

In [ ]:
# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use the executor to map the scrape_page function to the page numbers
    results = executor.map(scrape_page, range(1, numOfPages))

# Flatten the results into a single list
data = [item for sublist in results for item in sublist]

In [ ]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

nan_value = float("NaN") 
df.replace("", nan_value, inplace=True) 
df.dropna(how='all', axis=1, inplace=True) 

# Write the DataFrame to an Excel file
df.to_excel('output2_threaded.xlsx', index=False)

In [ ]:
# Close the driver
driver.quit()